# Automatic generation of image-captions

![alt text](model.png)

- Today's tutorial aims at showing you the capabilities that neural networks have when dealing with images and language. 
- We will see how to use some pre-trained neural networks to program our image-caption generator: we will create a simple system which given any image as input, will output a brief description of the image. 
- This system will consist of two neural networks, the first one will be responsible for dealing with the visual part of the problem (called the encoder), while the second one with the linguistic one (called the decoder). 
- A part of your work is also to deal with any missing library, using `conda install` or `pip install` whatever is missing

We will **not train these neural networks ourselves**, therefore you will need to have downloaded the following files: 

- `encoder.pkl` : a pickle file containing the weights of the encoder
- `encoder.pkl` : a pickle file containing the weights of the decoder
- `vocabulary.pkl` : a pickle file containing the vocabulary used
 


In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import transforms
import torchvision.models as models
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Load an image we want to generate a caption for

- Write a custom function which given a path to an image loads it with the PIL library (see imports above and https://he-arc.github.io/livre-python/pillow/index.html)
- The image should be resized to a 224x224 pixels image
- Explore the type of the object which gets loaded, do you think it is possible to use such an object as input to a neural network? Why/Why not?

In [ ]:
# Your code goes here


## Convert the loaded image to a tensor

An `Image` object cannot be used as input for a neural network. We need in fact to convert it to a tensor and normalize the values of its pixels

- Write a custom function that gets a PIL Image as input, normalizes its pixels and converts the image to a tensor which can be used as input for a neural network. For this, use the `torchvision` library, in particular its `transforms` module, and the `Compose`, `ToTensor` and `Normalize` classes, among others. 
- **Warning**: You should use `ImageNet`'s means and standard deviations to normalize each channel, look what this means and code it!
- The size of the output tensor should be `(1, 3, 244, 244)`

In [ ]:
# Your code goes here


## Load the vocabulary of words which will be used for creating a caption

The neural network will require a pool of words to choose from to come up with an appropriate caption for an image. 

- Write a function that loads the pickle file and returns the "vocabulary" object
- Look at the attributes contained in this "vocabulary". What are those ? What do they do ?

In [ ]:
# Your code goes here


## Create the neural network architecture 

Here we create the computational graph of the neural architecture which will get an image as input and outputs an appropriate caption.

- For today, we provide the code for this. You won't understand most of it, but a good exercice to learn is not "reverse engineer" it: try parts of the code, look at the documentation, to try understand what this code does
- The neural network consists of two different parts: an **encoder**, which uses a ResNet-152 architecture (it's a big convolutional network) and a **decoder**, which uses a simple LSTM (a kind of recurrent neural network).

Note that the code below only create the computational graphs of the networks architectures, but they are not trained at all, and are kind of empty shells.

In [ ]:
class EncoderCNN(nn.Module):

    def __init__(self, embed_size):
        """Loads the pretrained ResNet-152 but replaces the fully-connected layer.
        """
        super(EncoderCNN, self).__init__()
        resnet = models.resnet152(pretrained=True)
        # Delete the last fully-connected layer
        modules = list(resnet.children())[:-1]     
        self.resnet = nn.Sequential(*modules)
        self.linear = nn.Linear(resnet.fc.in_features, embed_size)
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)
        
    def forward(self, images):
        """Extract feature vectors from input images."""
        with torch.no_grad():
            features = self.resnet(images)
        features = features.reshape(features.size(0), -1)
        features = self.bn(self.linear(features))
        return features
    
def build_encoder(embedding_size):
    # Use eval mode (because of batch-normalization)
    encoder = EncoderCNN(embedding_size).eval()  
    encoder = encoder.to(device)
    return encoder 

embedding_size = 256
encoder = build_encoder(embedding_size)

In [ ]:
encoder

In [ ]:
encoder

In [ ]:
class DecoderRNN(nn.Module):

    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, max_seq_length=20):
        """Set the hyper-parameters and build the layers."""
        super(DecoderRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.max_seg_length = max_seq_length
        
    def forward(self, features, captions, lengths):
        """Decode image feature vectors and generates captions."""
        embeddings = self.embed(captions)
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True) 
        hiddens, _ = self.lstm(packed)
        outputs = self.linear(hiddens[0])
        return outputs
    
    def sample(self, features, states=None):
        """Generate captions for given image features using greedy search."""
        sampled_ids = []
        inputs = features.unsqueeze(1)
        for i in range(self.max_seg_length):
            hiddens, states = self.lstm(inputs, states)          # hiddens: (batch_size, 1, hidden_size)
            outputs = self.linear(hiddens.squeeze(1))            # outputs:  (batch_size, vocab_size)
            _, predicted = outputs.max(1)                        # predicted: (batch_size)
            sampled_ids.append(predicted)
            inputs = self.embed(predicted)                       # inputs: (batch_size, embed_size)
            inputs = inputs.unsqueeze(1)                         # inputs: (batch_size, 1, embed_size)
        sampled_ids = torch.stack(sampled_ids, 1)                # sampled_ids: (batch_size, max_seq_length)
        return sampled_ids


def build_decoder(embedding_size, hidden_size, vocab, num_layers):
    decoder = DecoderRNN(embedding_size, hidden_size, len(vocab["idx2word"]), num_layers)
    decoder = decoder.to(device)
    return decoder


hidden_size = 512
num_layers = 1
decoder = build_decoder(embedding_size, hidden_size, vocab, num_layers)

In [ ]:
decoder

## Load the networks weights within the computational graph

The neural networks created above are empty shells: they are not trained, meaning that the captions which we would obtain as output would be garbage. 
We need to load the appropriate weights within the computation graph. 

- Write a function which given a computational graph loads and the already trained weights that you have already downloaded. Pickle files (`encoder-*.pkl` and `decoder-*.pkl` files) can be loaded with `torch.load`. The files contain the `state_dict` or the networks, that can be passed through the `load_state_dict` method.

In [ ]:
# Your code goes here


## Extract meaningful information from the encoder

The encoder aims at converting the input image into a feature vector that represents the original image in a "meaningful" way. This corresponds to a *forward pass* in the encoder network. 

- Write a function that gets as input an image, the encoder-network and returns a feature vector.

- What is the difference between the extracted features and the original input of the network?

In [ ]:
# Your code goes here


## Match the extracted image features with a potential caption

Now, we want to use the features extracted from the image by the encoder as input for the decoder. The goal is to get from the decoder a set of words which corresponds to a relevant caption of the image. 

- Write a function which uses the decoder and the extracted image features to returns what will be later the words of the caption

**Hint**: the decoder will call the `.sample()` method defined above. Also, the output should no longer be a `pytorch` tensor, but rather a `numpy` array

In [ ]:
# Your code goes here


## Convert the output of the decoder to meaningful words

The decoder network does not output directly words but a list of integerers instead.
These integers are actually indexes for words from the vocabulary you loaded before.

- Write a function that converts the output of the decoder into meaningful words using the vocabulary
- If you meet the word `<end>` then you should stop the sentence
- Try to do this in the most Pythonic way possible (using the modules from the standard library, such as `itertools`)

In [ ]:
# Your code goes here


## Putting all this together

Now, you are able to load an image and generated a captioning of it.

- Write a function that takes as input an image and returns a readable caption about it. This function should take as input both the encoder and decoder (with loaded weights) and the vocabulary. Of course, you can use in this function some of the functions defined above. Try to avoid the use of objects from the global namespace (pass all useful objects as arguments). It's a good practice to follow, in other to avoid bugs.
- Try your function on several images ! 

In [ ]:
# Your code goes here


## Just for fun: try on any image (using your webcam)

In [ ]:
# Might require pip install opencv-python
import cv2

def camera_grab(camera_id=0):
    """This function captures an image through the webcam of your laptop (if accessible)
    """
    camera = cv2.VideoCapture(camera_id)
    try:
        for i in range(10):
            snapshot_ok, image = camera.read()
        if not snapshot_ok:
            raise RuntimeError("Could not access camera")
    finally:
        camera.release()
    return image

In [ ]:
image = camera_grab()
plt.figure(figsize=(8, 8))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')

In [ ]:
# Write the image on disk
plt.imsave("webcam.jpg", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

# Read again the image
image = load_image("webcam.jpg")

# Your code goes here: call your fonction to generate the caption here
